In [ ]:
!pip install -U sentence-transformers

In [4]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import files #uploading ds.
uploaded = files.upload()
filename = list(uploaded.keys())[0]
dataset = pd.read_csv(filename)
dataset['job_description'] = dataset['job_description'].astype(str)
dataset['resume'] = dataset['resume'].astype(str)

Saving resume_job_matching_dataset.csv to resume_job_matching_dataset.csv


In [6]:
dataset['job_description'] = dataset['job_description'].astype(str) #ds col originally not str.
dataset['resume'] = dataset['resume'].astype(str)

In [ ]:
from sentence_transformers import SentenceTransformer, util #to get semantic embeddings.

model = SentenceTransformer("all-MiniLM-L6-v2")  #384 dim
resume_embeddings = model.encode(dataset['resume'].tolist(), convert_to_numpy=True)
job_embeddings = model.encode(dataset['job_description'].tolist(), convert_to_numpy=True)


In [9]:
np.savez_compressed("embeddings.npz", job=job_embeddings, resume=resume_embeddings) #saving sbert embeddings.

In [17]:
data = np.load('embeddings.npz')
resume_embs = data['resume']
job_embs = data['job']
labels=dataset['match_score']

In [18]:

X = np.hstack([resume_embs, job_embs])  # 2*384
y = labels


In [19]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1
)
model.fit(X_train, y_train)

preds = model.predict(X_test)
mse = mean_squared_error(y_test, preds)
print("mse")


MSE: 0.7469


In [20]:
import joblib

# saving the model.
joblib.dump(model, 'xgboost_model.pkl')


['xgboost_model.pkl']